# Detecting Couple candlestick traps by using NEAT

In [24]:
## As similar mix-1 but no hidden input

### Import Library

In [25]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns
import os

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import neat

### Load Price Data

In [27]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir.parent) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print(csv_file)
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [28]:
data = dataset.copy()

In [29]:
data = data[data.index > '2020-11-01 00:00:00']

In [30]:
# len(data[(data.index > '2025-02-14 00:00:00') & (data.index < '2025-02-14 23:00:00')])

In [31]:
def set_condition_1(r):
    cond = ''
    if r['Open'] < r['Close'] <= r['High'] - 0.1:
        # Xanh va co bong nen tren
        cond = 'long'
    return cond


def set_condition_2(r):
    cond = ''
    if r['Open'] < r['Close'] == r['High'] and r['High'] > r['high_s1']:
        # Xanh va khong co bong nen tren
        cond = 'long'
    return cond
    

def get_couple_candleticks_signal(r):
    signal = ''
    if 100 * r.name.hour + r.name.minute >= 1425:
        return ''
    if r['condition_1'] == 'long' and r['condition_2'] == 'long':
        signal = 'long'
    return signal

In [32]:
# Couple candlesticks signal
data['low_s1'] = data['Low'].shift(1)
data['high_s1'] = data['High'].shift(1)
data['condition_1'] = data.apply(lambda r: set_condition_1(r), axis=1)
data['condition_1'] = data['condition_1'].shift(1)
data['condition_2'] = data.apply(lambda r: set_condition_2(r), axis=1)
data['signal'] = data.apply(lambda r: get_couple_candleticks_signal(r), axis=1)

In [33]:
LOOK_BACK = 51 # 1 day analytics
data['Higher_High'] = (data['High'] > data['High'].shift(1)).astype(int)
data['Lower_Low'] = (data['Low'] < data['Low'].shift(1)).astype(int)
data['Higher_High_Count'] = data['Higher_High'].rolling(LOOK_BACK).sum()
data['Lower_Low_Count'] = data['Lower_Low'].rolling(LOOK_BACK).sum()
data['Green_cs'] = (data['Close'] > data['Open']).astype(int)
data['Red_cs'] = (data['Close'] < data['Open']).astype(int)
data['Green_CS_Count'] = data['Green_cs'].rolling(LOOK_BACK).sum()
data['Red_CS_Count'] = data['Red_cs'].rolling(LOOK_BACK).sum()
data['Diff_Open_Close'] = data['Close'] - data['Open']
data['Diff_Open_Close_Sum'] = data['Diff_Open_Close'].rolling(LOOK_BACK).sum()

In [34]:
len(data[data.signal != ''])

1102

In [35]:
data[data.signal != '']

,Open,High,Low,Close,Volume,low_s1,high_s1,condition_1,condition_2,signal,Higher_High,Lower_Low,Higher_High_Count,Lower_Low_Count,Green_cs,Red_cs,Green_CS_Count,Red_CS_Count,Diff_Open_Close,Diff_Open_Close_Sum
Date,,,,,,,,,,,,,,,,,,,,
2020-11-04 10:05:00,908.2,909.0,908.1,909.0,1800,907.9,908.6,long,long,long,1,0,19.0,22.0,1,0,19.0,27.0,0.8,0.9
2020-11-04 10:15:00,910.2,911.5,909.8,911.5,3514,909.1,910.5,long,long,long,1,0,20.0,21.0,1,0,21.0,27.0,1.3,3.1
2020-11-04 10:35:00,911.2,912.5,910.8,912.5,2731,910.7,911.5,long,long,long,1,0,21.0,21.0,1,0,21.0,28.0,1.3,2.8
2020-11-04 11:00:00,913.5,914.2,912.8,914.2,1996,913.0,913.7,long,long,long,1,1,22.0,21.0,1,0,22.0,27.0,0.7,2.9
2020-11-06 13:40:00,904.5,906.7,904.4,906.7,2806,903.5,905.0,long,long,long,1,0,18.0,23.0,1,0,19.0,29.0,2.2,-11.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-05 13:50:00,1331.9,1333.4,1331.5,1333.4,5276,1330.6,1331.8,long,long,long,1,0,23.0,21.0,1,0,20.0,27.0,1.5,0.9
2025-02-06 09:55:00,1342.5,1344.3,1342.3,1344.3,4723,1341.5,1342.7,long,long,long,1,0,23.0,20.0,1,0,24.0,24.0,1.8,9.9
2025-02-11 13:25:00,1331.2,1332.5,1331.2,1332.5,3534,1329.6,1331.4,long,long,long,1,0,19.0,23.0,1,0,22.0,24.0,1.3,-4.3


## TRAP labeling

In [36]:
traps = []
for i, row in data.iterrows():
    if row['signal']:
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date + ' 14:30:00')]
        #
        data_at_end_day = data[data.index == current_date + ' 14:25:00']
        last_close = data_at_end_day.iloc[0]['Close']
        #
        if len(data_to_end_day[data_to_end_day.Low < row['Close'] - 3.5]) > 0 or row['Close'] > last_close:
            traps.append(1)
        else:
            traps.append(0)
    else:
        traps.append('')

In [37]:
data['trap'] = traps

In [38]:
data.dropna(inplace=True)

In [39]:
signal_data = data[data.signal != '']
signal_data.dropna(inplace=True)

In [40]:
len(signal_data)

1102

## Features

In [41]:
X = signal_data[['Higher_High_Count', 'Lower_Low_Count', "Green_CS_Count", 'Red_CS_Count', 'Diff_Open_Close_Sum', "trap"]]
# Train-Test Split
X_train = X[X.index < '2024-07-01 00:00:00']
X_test = X[X.index > '2024-07-01 00:00:00']
X_train['trap'] = X_train['trap'].astype(int)

In [42]:
len(X_train)

959

In [43]:
len(X_train[X_train.trap == 0])

354

In [44]:
X_none_trap = X_train[X_train.trap == 0]
balanced_X_train = pd.concat([X_train, X_none_trap], ignore_index=True)

In [45]:
len(balanced_X_train)

1313

In [46]:
len(balanced_X_train[balanced_X_train.trap == 0]) / len(balanced_X_train)

0.5392231530845393

In [83]:
len(X_test)

143

In [47]:
X_test

,Higher_High_Count,Lower_Low_Count,Green_CS_Count,Red_CS_Count,Diff_Open_Close_Sum,trap
Date,,,,,,
2024-07-02 09:05:00,26.0,25.0,21.0,21.0,8.6,0
2024-07-02 11:25:00,37.0,20.0,27.0,18.0,20.2,0
2024-07-03 10:30:00,25.0,19.0,23.0,22.0,4.9,0
2024-07-03 13:25:00,23.0,18.0,22.0,21.0,6.3,0
2024-07-04 10:55:00,28.0,17.0,26.0,21.0,10.2,1
...,...,...,...,...,...,...
2025-02-05 13:50:00,23.0,21.0,20.0,27.0,0.9,0
2025-02-06 09:55:00,23.0,20.0,24.0,24.0,9.9,1
2025-02-11 13:25:00,19.0,23.0,22.0,24.0,-4.3,1


In [50]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        genome.fitness = 4.0
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        for move_index, row in balanced_X_train.iterrows():
            inputs = [row['Higher_High_Count'], row['Lower_Low_Count'], row['Green_CS_Count'], row['Red_CS_Count'], row['Diff_Open_Close_Sum']]
            expected_output = row['trap']
            output = net.activate(inputs)
            genome.fitness -= (output[0] - expected_output) ** 2
            if genome.fitness < -200:
                break


def run(config_file):
    # Load configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    # p.add_reporter(neat.StdOutReporter(True))
    # stats = neat.StatisticsReporter()
    # p.add_reporter(stats)

    # Run for up to 100 generations.
    winner = p.run(eval_genomes, 100)

    # Display the winning genome.
    print('\nBest genome:\n{!s}'.format(winner))
    return neat.nn.FeedForwardNetwork.create(winner, config)

In [51]:
config_path = os.path.join(current_dir, 'style-mix-3.cfg')
best_brain = run(config_path)


Best genome:
Key: 1603
Fitness: -200.00000000000003
Nodes:
	0 DefaultNodeGene(key=0, bias=0.729219333932245, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-4, 0), weight=-0.7027739443848795, enabled=True)
	DefaultConnectionGene(key=(-3, 0), weight=2.3059137101741527, enabled=True)


In [52]:
best_brain

In [53]:
# Show output of the most fit genome against training data.
outputs = []
for i, row in X_test.iterrows():
    inputs = [row['Higher_High_Count'], row['Lower_Low_Count'], row['Green_CS_Count'], row['Red_CS_Count'], row['Diff_Open_Close_Sum']]
    expected_output = row['trap']
    output = best_brain.activate(inputs)
    outputs.append(round(output[0]))
    # print("input {!r}, expected output {!r}, got {!r}".format(inputs, expected_output, output))

In [54]:
expected_outputs = X_test['trap'].to_list()
# Evaluate Performance
print("Accuracy:", accuracy_score(expected_outputs, outputs))

Accuracy: 0.6013986013986014
